In [ ]:
import sys
from pathlib import Path
from os import getcwd
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.systems import CasosConfirmados, Notifica

In [ ]:
import pandas as pd
from datetime import date, timedelta
from os.path import join
from epiweeks import Week

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')

### 1. Coeficiente de Incidência

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}", compress=True)

In [ ]:
init = hoje - timedelta(6)
end = hoje
print(init, end)

In [ ]:
# init = pd.to_datetime(Week.thisweek().startdate() - timedelta(7))
# end = pd.to_datetime(Week.thisweek().enddate() - timedelta(7))
# print(init, end)

In [ ]:
semana = cc.df.loc[cc.df['uf_residencia']=='PR']
semana = semana.loc[(semana['data_diagnostico'] >= init) & (semana['data_diagnostico'] <= end)]

In [ ]:
casos_semana = semana.join(cc.tables['municipios'].set_index('ibge')[['rs','regional','municipio','populacao']], on='ibge_residencia').groupby(['rs','regional','municipio'])[['paciente']].count().rename(columns={'paciente':'casos'})
# casos_semana.loc[('','','PR'),:] = casos_semana.sum()
casos_semana

In [ ]:
pop_reg = cc.tables['municipios'].loc[cc.tables['municipios']['uf']=='PR'].groupby(['rs','regional','municipio'])[['populacao']].sum()
# pop_reg.loc[('','','PR'),:] = pop_reg.sum()

In [ ]:
casos_semana = casos_semana.reindex(pop_reg.index).fillna(0)
coef_reg = casos_semana / pop_reg.values * 100000
coef_reg = coef_reg.rename(columns={'casos':'value'})

In [ ]:
df = pop_reg.join(casos_semana).join(coef_reg).reset_index()
df.loc[(df['casos']<200) & (df['value']<10),'label'] = 'L'
df.loc[(df['casos']<200) & (df['value']>10) & (df['value']<20),'label'] = 'M'
df.loc[(df['casos']<200) & (df['value']>20),'label'] = 'H'

df.loc[(df['casos']>=200) & (df['value']<10),'label'] = 'M'
df.loc[(df['casos']>=200) & (df['value']>=10),'label'] = 'H'

df

In [ ]:
df2 = df.reset_index().groupby(['rs','regional','label'])['casos'].count().unstack('label')
df2.loc[('','Total'),:] = df2.sum(axis=0)
df2 = df2.fillna(0).astype(int)
df2

In [ ]:
df.to_excel(join(default_output, 'dash', 'coef_regional.xlsx'), index=False)

### 2. Positividade de Testes

In [ ]:
today = date.today()
seven_days_ago = today - timedelta(6)

In [ ]:
# gal_2020 = pd.read_pickle(join(path_file, 'banco_gal_2020-bz2.pkl'), compression='bz2')
# gal_2020 = gal_2020#[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]

In [ ]:
# gal_2021 = pd.read_pickle(join(path_file, 'banco_gal_2021-bz2.pkl'), compression='bz2')
# gal_2021 = gal_2021#[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]

In [ ]:
gal_complemento = pd.read_csv(join(path_file, 'consulta-gal-complemento.csv'), sep = ';')#, usecols=['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao'])

In [ ]:
# gal = pd.concat([gal_2020, gal_2021, gal_complemento])
gal = pd.concat([gal_complemento])

In [ ]:
boletim = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2']
boletim = boletim.loc[boletim['unidade_requisitante_uf'] == 'PARANA']
boletim = boletim.loc[boletim['regional'] != 'Outra UF']

In [ ]:
init = str(seven_days_ago) + " 00:00:00"
print(init)
boletim = boletim.loc[boletim['data_liberacao'] >= init]

In [ ]:
descartado = boletim.loc[boletim['classificacao'] == 'Descartado']
descartado = descartado[['id', 'regional']].groupby(['regional']).count().reset_index()
descartado = descartado.rename(columns={'id':'Descartado'})
# descartado

In [ ]:
confirmado = boletim.loc[boletim['classificacao'] == 'Confirmado']
confirmado = confirmado[['id', 'regional']].groupby(['regional']).count().reset_index()
confirmado = confirmado.rename(columns={'id':'Confirmado'})
confirmado

In [ ]:
tabela = pd.DataFrame()
tabela = pd.merge(left=descartado, right=confirmado, how='left', on=['regional']).fillna(0)


tabela.loc[tabela['regional'] == '1ª R.S. - Paranaguá', 'regional'] = 'RS01'
tabela.loc[tabela['regional'] == '2ª R.S. - Metropolitana', 'regional'] = 'RS02'
tabela.loc[tabela['regional'] == '3ª R.S. - Ponta Grossa', 'regional'] = 'RS03'
tabela.loc[tabela['regional'] == '4ª R.S. - Irati', 'regional'] = 'RS04'
tabela.loc[tabela['regional'] == '5ª  R.S. - Guarapuava', 'regional'] = 'RS05'
tabela.loc[tabela['regional'] == '6ª R.S. - União da Vitória', 'regional'] = 'RS06'
tabela.loc[tabela['regional'] == '7ª R.S. - Pato Branco', 'regional'] = 'RS07'
tabela.loc[tabela['regional'] == '8ª R.S. - Francisco Beltrão', 'regional'] = 'RS08'
tabela.loc[tabela['regional'] == '9ª R.S. - Foz do Iguaçu', 'regional'] = 'RS09'
tabela.loc[tabela['regional'] == '10ª R.S. - Cascavel', 'regional'] = 'RS10'
tabela.loc[tabela['regional'] == '11ª R.S. - Campo Mourão', 'regional'] = 'RS11'
tabela.loc[tabela['regional'] == '12ª R.S. - Umuarama', 'regional'] = 'RS12'
tabela.loc[tabela['regional'] == '13ª R.S. - Cianorte', 'regional'] = 'RS13'
tabela.loc[tabela['regional'] == '14ª R.S. - Paranavaí', 'regional'] = 'RS14'
tabela.loc[tabela['regional'] == '15ª R.S. - Maringá', 'regional'] = 'RS15'
tabela.loc[tabela['regional'] == '16ª R.S. - Apucarana', 'regional'] = 'RS16'
tabela.loc[tabela['regional'] == '17ª R.S. - Londrina', 'regional'] = 'RS17'
tabela.loc[tabela['regional'] == '18ª R.S. - Cornélio Procópio', 'regional'] = 'RS18'
tabela.loc[tabela['regional'] == '19ª R.S. - Jacarezinho', 'regional'] = 'RS19'
tabela.loc[tabela['regional'] == '20ª R.S. - Toledo', 'regional'] = 'RS20'
tabela.loc[tabela['regional'] == '21ª R.S. - Telêmaco Borba', 'regional'] = 'RS21'
tabela.loc[tabela['regional'] == '22ª R.S. - Ivaiporã', 'regional'] = 'RS22'


tabela = tabela.sort_values('regional')

In [ ]:
tabela.loc['TOTAL PR', ['Descartado', 'Confirmado']] = tabela[['Descartado', 'Confirmado']].sum()

tabela['TOTAL'] = tabela['Descartado'] + tabela['Confirmado']
tabela['POSITIVIDADE'] = tabela['Confirmado'] / tabela['TOTAL']

In [ ]:
tabela['label'] = ''
tabela.loc[tabela['POSITIVIDADE']<0.05,'label'] = 'L'
tabela.loc[(tabela['POSITIVIDADE']>=0.05)&(tabela['POSITIVIDADE']<0.08),'label'] = 'M'
tabela.loc[(tabela['POSITIVIDADE']>=0.08)&(tabela['POSITIVIDADE']<0.1),'label'] = 'S'
tabela.loc[tabela['POSITIVIDADE']>=0.1,'label'] = 'H'
tabela

In [ ]:
tabela.to_excel(join(default_output, 'dash', 'positividade_regional.xlsx'))